### Selenium의 스크롤링
    - body = driver.find_element_by_css_selector("body") 해당 element 지정시 body 태그를 이용
    - Page up : body.send_keys(Keys.PAGE_UP)
    - Page down : body.send_keys(Keys.PAGE_DOWN)
    - End : body.send_keys(Keys.END)
    - height = driver.excute_script("return document.body.scrollHeight")

## 유튜브 
    -제목, 조회수

In [77]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

import time
import pandas as pd

In [33]:
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get("https://www.youtube.com/c/hahahaYouTube/videos")

In [45]:
from bs4 import BeautifulSoup as bs
body = driver.find_element_by_css_selector("body")

while True:
    # END 키를 누르기전과 누른후의 html을 비교
    last = bs(driver.page_source,'html.parser')
    for i in range(10):
        body.send_keys(Keys.END)
    new = bs(driver.page_source, 'html.parser')
    
    if last == new:
        break


In [65]:
from tokenize import Double


title = driver.find_elements_by_css_selector('h3.style-scope.ytd-grid-video-renderer>a.yt-simple-endpoint.style-scope.ytd-grid-video-renderer')
view = driver.find_elements_by_css_selector('div.style-scope.ytd-grid-video-renderer>span:nth-child(1).style-scope.ytd-grid-video-renderer')

title_list =[]
view_list =[]

for i in title:
    title_list.append(i.text)
for i in view:
    a = i.text
    a = a.replace('조회수 ','')
    a = a.replace('회','')
    if a[-1]=='만':
        a = a.replace('만','')
        a = (float(a) * 10000)
    else:
        float(a)
    view_list.append(int(a))

data = {'제목' : title_list, '조회수' : view_list}
yt = pd.DataFrame(data)
yt

,제목,조회수
0,딸을 대하는 방법이 다 다른 세 엄마 고양이들,120000
1,고양이 때문에 드라마를 볼 수가 없다,270000
2,태풍이 왔었지만 모두 무사하다는 영상,290000
3,하하하 식당,170000
4,촬영을 포기할 수 밖에 없었다,260000
...,...,...
524,배스잡아 고양이 밥주기,250000
525,"큰쏘가리, 황쏘가리 수중촬영",80000
526,어린쏘가리 수중촬영,66000
527,고양이 뱀사냥,1550000


In [66]:
yt = yt.sort_values(by=['조회수'], ascending = False)

yt

,제목,조회수
232,얼음 속 잉어를 노리는 고양이,18610000
359,국내최초 양어장vlog(특별출연 길막이와 삼색이),9680000
280,삼색이 때문에 낚시를 못하겠다,8950000
321,방수 솜방망이,7150000
502,고양이의 잉어먹방!,7140000
...,...,...
0,딸을 대하는 방법이 다 다른 세 엄마 고양이들,120000
522,고양이 밥으로 배스 끓여주기,110000
373,공 속에 카메라를 넣고 던져보았다,96000
525,"큰쏘가리, 황쏘가리 수중촬영",80000


In [67]:
yt = yt.reset_index(drop=True)
yt

,제목,조회수
0,얼음 속 잉어를 노리는 고양이,18610000
1,국내최초 양어장vlog(특별출연 길막이와 삼색이),9680000
2,삼색이 때문에 낚시를 못하겠다,8950000
3,방수 솜방망이,7150000
4,고양이의 잉어먹방!,7140000
...,...,...
524,딸을 대하는 방법이 다 다른 세 엄마 고양이들,120000
525,고양이 밥으로 배스 끓여주기,110000
526,공 속에 카메라를 넣고 던져보았다,96000
527,"큰쏘가리, 황쏘가리 수중촬영",80000


## 카카오 맵
    - page 수 마다 URL이 달라지지 않음
    - 페이지를 넘겨가면서 데이터를 수집
    - 장소,별점 수집

In [103]:
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get("https://map.kakao.com/")

a = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')
a.click
a.send_keys('맛집')
a.send_keys(Keys.ENTER)

a = driver.find_element_by_xpath('/html/body/div[10]/div/div/img')
actions = ActionChains(driver)
actions.move_to_element(a)
actions.click(a)
actions.perform()

a = driver.find_element_by_css_selector('#info\.body')
a.click()

a = driver.find_element_by_css_selector('#info\.search\.place\.more')
a.click()

In [138]:
title_list=[]
rating_list=[]
b=0
while True:
    for i in range(1,6):
        page = driver.find_element_by_css_selector(f'#info\.search\.page\.no{i}')
        page.click()
        time.sleep(1)
        title = driver.find_elements_by_css_selector('strong > a.link_name')
        rating = driver.find_elements_by_css_selector('span.score > em')
        
        for i in title:
            title_list.append(i.text)
        for i in rating:
            rating_list.append(i.text)
    b=b+1
    if b == 2:
        break
    more = driver.find_element_by_css_selector('#info\.search\.page\.next')
    more.click()
    time.sleep(1)

In [139]:
data = {'음식점' : title_list, '평점' : rating_list}
fm = pd.DataFrame(data)
fm

,음식점,평점
0,광고크래버대게나라 서초점,4.0
1,텍사스데브라질 센트럴시티점,3.3
2,산들해 반포점,3.5
3,JW메리어트호텔서울 플레이버즈,4.1
4,한신치킨호프,4.2
...,...,...
146,BBQ 반포중앙점,3.0
147,우와정,4.0
148,베이크 신세계백화점 강남점,3.7
149,BHC치킨 반포한강공원1호점,1.1


In [140]:
name_list = []
review_list = []
i=2

for n in range(2,18):
    try:
        name = driver.find_element_by_css_selector(f'li:nth-child({n}) > div.head_item.clickArea > strong > a.link_name')
        review = driver.find_element_by_css_selector(f'li:nth-child({n}) > div.rating.clickArea > span.score > em')

        name_list.append(name.text)
        review_list.append(review.text)
    except:
        continue


a = driver.find_element_by_css_selector(f'#info\.search\.page\.no{i}')
a.click()
i =i+1
time.sleep(1)
b=1

while True:
    for n in range(1,17):
        try:
            name = driver.find_element_by_css_selector(f'li:nth-child({n}) > div.head_item.clickArea > strong > a.link_name')
            review = driver.find_element_by_css_selector(f'li:nth-child({n}) > div.rating.clickArea > span.score > em')

            name_list.append(name.text)
            review_list.append(review.text)
        except:
            continue
    b =  b+1
    if (i-1) % 5 == 0:
        a = driver.find_element_by_css_selector(f'#info\.search\.page\.next')
        a.click()
        i = 2
        time.sleep(1)
    else: 
        a = driver.find_element_by_css_selector(f'#info\.search\.page\.no{i}')
        a.click()
        i =i+1
        time.sleep(1)
    if b == 10:
        break
    

In [141]:
data = {'음식점' : name_list, '평점' : review_list}
fm = pd.DataFrame(data)
fm

,음식점,평점
0,텍사스데브라질 센트럴시티점,3.3
1,산들해 반포점,3.5
2,JW메리어트호텔서울 플레이버즈,4.1
3,한신치킨호프,4.2
4,데블스도어 센트럴시티점,3.5
...,...,...
145,BBQ 반포중앙점,3.0
146,우와정,4.0
147,베이크 신세계백화점 강남점,3.7
148,BHC치킨 반포한강공원1호점,1.1
